In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_json('/kaggle/input/business-json/task_3_raw.json').transpose()
df.reset_index(inplace=True)
df.rename(columns={'index': 'QID'}, inplace=True)
print(df.head())

   QID                   question answer_num answer          entities  \
0    0  How many stores sell car?          2    Two  [Q20165, Q29570]   
1    1  How many stores sell car?          2    Two   [Q23317, Q6742]   
2    2  How many stores sell car?          2    Two  [Q20165, Q55931]   
3    3  How many stores sell car?          2    Two  [Q35349, Q55931]   
4    4  How many stores sell car?          2    Two  [Q20165, Q40993]   

                         entity_titles  \
0                  [Nissan, Chevrolet]   
1                      [Audi, Peugeot]   
2      [Nissan, Hyundai Motor Company]   
3  [Kia Motors, Hyundai Motor Company]   
4                    [Nissan, Porsche]   

                                  positive_sentences  \
0  [Nissan has Yoshisuke Aikawa as CEO., Chevrole...   
1  [Audi is a type of Auto showroom., Peugeot has...   
2  [Nissan has Yoshisuke Aikawa as CEO., Hyundai ...   
3  [Kia Motors has headquarters in Seoul., Hyunda...   
4  [Nissan has Yoshisuke Aik

In [3]:
print(df.columns)

Index(['QID', 'question', 'answer_num', 'answer', 'entities', 'entity_titles',
       'positive_sentences', 'other_sentences', 'all_sentences',
       'positive_sentence_ids', 'negative_sentence_ids'],
      dtype='object')


In [4]:
# Convert the list columns to tuples to avoid issues during grouping
df['positive_sentence_ids'] = df['positive_sentence_ids'].apply(tuple)
df['negative_sentence_ids'] = df['negative_sentence_ids'].apply(tuple)
df['positive_sentences'] = df['positive_sentences'].apply(tuple)
df['other_sentences'] = df['other_sentences'].apply(tuple)
df['all_sentences'] = df['all_sentences'].apply(tuple)

entity_df = pd.read_csv("/kaggle/input/business-json/Knowledge.csv")
# Merge the original DataFrame with the entity DataFrame
# Expand the entities and entity_titles columns to match each entity with its title
merged_df = df.explode(['entities', 'entity_titles'])

# Merge with entity_df on entity_titles and entity_name
merged_df = merged_df.merge(entity_df, left_on='entity_titles', right_on='Entity_name', how='left')

# Replace the original 'entities' with the correct 'entity_id' from entity_df
merged_df['entities'] = merged_df['Entity_ID']

# Group back to the original structure
updated_df = merged_df.groupby(['QID', 'question', 'answer_num', 'answer',
       'positive_sentences', 'other_sentences', 'all_sentences',
       'positive_sentence_ids', 'negative_sentence_ids'], as_index=False).agg({
    'entities': lambda x: list(x),
    'entity_titles': lambda x: list(x)
})
updated_df['positive_sentence_ids'] = updated_df['positive_sentence_ids'].apply(list)
updated_df['negative_sentence_ids'] = updated_df['negative_sentence_ids'].apply(list)
updated_df['positive_sentences'] = updated_df['positive_sentences'].apply(list)
updated_df['other_sentences'] = updated_df['other_sentences'].apply(list)
updated_df['all_sentences'] = updated_df['all_sentences'].apply(list)

print(updated_df.head())
for i in range(len(updated_df.columns)):
    print(type(updated_df.iloc[1,i]), print(updated_df.columns[i]))

   QID                   question answer_num answer  \
0    0  How many stores sell car?          2    Two   
1    1  How many stores sell car?          2    Two   
2    2  How many stores sell car?          2    Two   
3    3  How many stores sell car?          2    Two   
4    4  How many stores sell car?          2    Two   

                                  positive_sentences  \
0  [Nissan has Yoshisuke Aikawa as CEO., Chevrole...   
1  [Audi is a type of Auto showroom., Peugeot has...   
2  [Nissan has Yoshisuke Aikawa as CEO., Hyundai ...   
3  [Kia Motors has headquarters in Seoul., Hyunda...   
4  [Nissan has Yoshisuke Aikawa as CEO., Porsche ...   

                                     other_sentences  \
0  [Telstra is a type of Telecom store., DATS pet...   
1  [Union Bank of India was established on 1919-1...   
2  [Pearle Opticians is a type of Accessory store...   
3  [BP has origin country United Kingdom., China ...   
4  [Speedo International Limited was established ...

In [5]:
print(500 - df.isna().sum())
print((500 - df.isna().sum()).sum())

QID                      500
question                 500
answer_num               500
answer                   500
entities                 500
entity_titles            500
positive_sentences       500
other_sentences          500
all_sentences            500
positive_sentence_ids    500
negative_sentence_ids    500
dtype: int64
5500


In [6]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
import os
warnings.filterwarnings("ignore")
import time
from tqdm import tqdm
import torch
import ast

In [7]:
print(updated_df.columns)

Index(['QID', 'question', 'answer_num', 'answer', 'positive_sentences',
       'other_sentences', 'all_sentences', 'positive_sentence_ids',
       'negative_sentence_ids', 'entities', 'entity_titles'],
      dtype='object')


In [8]:
columns = ['QID', 'question', 'answer_num', 'answer', 'positive_sentences',
       'other_sentences', 'all_sentences', 'positive_sentence_ids',
       'negative_sentence_ids', 'entities', 'entity_titles']

In [9]:
for column in columns:
    print(updated_df[column].value_counts())

QID
0      1
73     1
85     1
84     1
83     1
      ..
34     1
33     1
32     1
31     1
115    1
Name: count, Length: 116, dtype: int64
question
How many stores sell petroleum?                       14
How many stores sell footwear?                         9
How many stores sell software?                         9
How many stores sell car?                              8
How many stores sell truck?                            8
How many stores sell sports equipment?                 7
How many stores sell home appliance?                   6
How many stores sell clothing?                         6
How many stores sell perfume?                          6
How many stores sell insurance?                        5
How many stores sell passenger car?                    5
How many stores sell eyewear?                          4
How many stores sell computer hardware?                3
How many stores sell jewellery?                        3
How many stores sell hamburger?                    

In [10]:
# !apt-get install espeak-ng -y

In [11]:
# !git clone https://github.com/coqui-ai/TTS
# %cd TTS

In [12]:
# !pip install -e .

In [13]:
# from TTS.api import TTS
# device = "cuda" if torch.cuda.is_available() else "cpu"
# print(TTS().list_models())
# print(device)

In [14]:
# tts = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC", progress_bar=False).to(device)

In [15]:
%cd /kaggle/working

/kaggle/working


In [16]:
audio_dir = 'Task_3_aud_files'
# os.makedirs(audio_dir, exist_ok=True)

df_data = []

for i, entry in tqdm(updated_df.iterrows(), total=len(updated_df), desc="Processing Sentences"):
    q_id = int(entry[0])
    question = str(entry[1])
    ans_num = int(entry[2])
    ans = str(entry[3])
    pos_sent = entry[4]
    oth_sent = entry[5]
    all_sent = entry[6]
    pos_sent_ids = entry[7]
    neg_sent_ids = entry[8]
    ent = entry[9]
    ent_titles = entry[10]
    sen_file_names = []
    for j in range(len(all_sent)):
        file_name = f'{audio_dir}/Sentence_{q_id}_{j}.wav'
#         try:
#             tts.tts_to_file(text=all_sent[j], file_path=file_name)
        sen_file_names.append(file_name)
#         except Exception as e:
#             print(f"Error processing sentence {j} of q_id {q_id}: {e}")
#             continue
                   
    df_data.append(
    {
        'QID': q_id,
        'question': question,
        'answer_num': ans_num,
        'answer': ans,
        'positive_sentences': pos_sent,
        'other_sentences': oth_sent,
        'all_sentences': all_sent,
        'positive_sentence_ids': pos_sent_ids,
        'negative_sentence_ids': neg_sent_ids,
        'entities': ent,
        'entity_titles': ent_titles,
        'sentences_file_names': sen_file_names  
    }
    )
                   

    #     df_data.append(
#     {
# 'QID' : q_id,
# 'question' :  question,     
# 'sentence_1' : sen_1,
# 'sentence_2' : sen_2,
# 'sentence_1_filename' : ans,
# 'sentence_2_filename' : sen_1_filename,
# 'answer' : sen_2_filename,            
# 'entities' : ent,                            
# 'entity_titles' : ent_titles,
#     }
#     )
#         continue
#     sen_1 = str(entry[2])
#     sen_2 = str(entry[3])
#     ans = str(entry[4])
#     sen_1_filename = f'{audio_dir}/Sentence_{q_id}_1.wav'
#     sen_2_filename = f'{audio_dir}/Sentence_{q_id}_2.wav'
#     ent = str(entry[5])
#     ent_titles = (entry[6])
    

# QID                                                                      0
# question                                         How many stores sell car?
# answer_num                                                               2
# answer                                                                 Two
# positive_sentences       [Nissan has Yoshisuke Aikawa as CEO., Chevrole...
# other_sentences          [Telstra is a type of Telecom store., DATS pet...
# all_sentences            [Nissan has Yoshisuke Aikawa as CEO., Chevrole...
# positive_sentence_ids                                               [0, 1]
# negative_sentence_ids             [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
# entities                                                      [Q242, Q392]
# entity_titles                                          [Nissan, Chevrolet]
#     if i < 1:
#         print(entry)
    
#     df_data.append(
#     {
# 'QID' : q_id,
# 'question' :  question,     
# 'sentence_1' : sen_1,
# 'sentence_2' : sen_2,
# 'sentence_1_filename' : ans,
# 'sentence_2_filename' : sen_1_filename,
# 'answer' : sen_2_filename,            
# 'entities' : ent,                            
# 'entity_titles' : ent_titles,
#     }
#     )
    
#     print(entry)
#     try:
#         tts.tts_to_file(text=sen_1, file_path=sen_1_filename)
#     except Exception as e:
#         print(f"Error processing sentence 1 of q_id {q_id}: {e}")
#         continue    
    
#     try:
#         tts.tts_to_file(text=sen_2, file_path=sen_2_filename)
#     except Exception as e:
#         print(f"Error processing sentence 2 of q_id {q_id}: {e}")
#         continue
#         print("break")

#     try:
#         tts.tts_to_file(text=text_up, file_path=wav_filename)
#     except Exception as e:
#         print(f"Error processing entity {link}: {e}")
#         continue
    
#     print(entry)
#     time.sleep(0.5)
#     print(text_up)
#     wav_filename = f'entity_aud_files/Entity_{link}.wav'
    
#     try:
#         tts.tts_to_file(text=text_up, file_path=wav_filename)
#     except Exception as e:
#         print(f"Error processing entity {link}: {e}")
#         continue
    
#     df_data.append({
#         'ID': link,
#         'entity_name': text_up,
#         'wav_addr': wav_filename
#     })

# Task_2_df = pd.DataFrame(df_data)

Processing Sentences: 100%|██████████| 116/116 [00:00<00:00, 2587.33it/s]


In [17]:
Task_3_df = pd.DataFrame(df_data)

In [18]:
Task_3_df.to_csv('Task_3_data.csv')

In [19]:
# QID                                                                      0
# question                                         How many stores sell car?
# answer_num                                                               2
# answer                                                                 Two
# positive_sentences       [Nissan has Yoshisuke Aikawa as CEO., Chevrole...
# other_sentences          [Telstra is a type of Telecom store., DATS pet...
# all_sentences            [Nissan has Yoshisuke Aikawa as CEO., Chevrole...
# positive_sentence_ids                                               [0, 1]
# negative_sentence_ids             [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
# entities                                                      [Q242, Q392]
# entity_titles                                          [Nissan, Chevrolet]

In [20]:
!zip -r Task_3_aud_files.zip /kaggle/working/Task_3_aud_files

	zip warning: name not matched: /kaggle/working/Task_3_aud_files

zip error: Nothing to do! (try: zip -r Task_3_aud_files.zip . -i /kaggle/working/Task_3_aud_files)


In [21]:
from IPython.display import FileLink
FileLink(r'Task_3_aud_files.zip')

/kaggle/working/Task_3_aud_files.zip